# Graph-Clear

In a graph-clear problem, we are given a floormap represented by an undirected graph $(N, E)$, where $N = \{ 0, ..., n - 1 \}$ is the set of nodes, and $E \subseteq N \times N$ is the set of edges.
Each node corresponds to a room, and edges are corridors connecting two rooms.
We want to clear intruders in the floor using robots.
At each time step, we can clear a node $i$ using $a_i$ robots to sweep the room and use $b_{ij}$ robots to block each incident edge $\{ i, j \}$.
At the beginning, all nodes are contaminated, i.e., potentially include intruders.
Even if a node is swept, if there exists a non-blocked path from a contaminated node to that node, it become contaminated again in the next time step.
Therefore, we may want to block edges that are not directly connected to the currently swept node.
We want to find a schedule over time steps to clear all nodes while minimizing the maximum number of robots used at a time.

## DP Formulation

It is proved that there exists an optimal schedule where an already swept node never becomes contaminated again.
We just need to clear a node one by one while blocking all edges connected to already swept nodes.
Let $C \subseteq N$ be the set of already swept nodes, and assume that $b_{ij} = 0$ if $\{ i, j \} \notin E$.
To clear node $c \in \overline{C} = N \setminus C$, we need to use $a_c$ robots to sweep $c$, $\sum_{i \in N} b_{ci}$ robots to block the edges incident to $c$, and $\sum_{i \in C} \sum_{j \in \overline{C} \setminus \{ c \}} b_{ij}$ robots to block the edges connected to already swept nodes.
Therefore,

$$
\begin{align}
    \text{compute } & V(\emptyset) \\
    & V(C) = \begin{cases}
        \min\limits_{c \in \overline{C}} \max\left\{ a_c + \sum\limits_{i \in N} b_{ci} + \sum\limits_{i \in C} \sum\limits_{j \in \overline{C} \setminus \{ c \}} b_{ij}, V(C \cup \{ c \}) \right\} & \text{if } C \neq N \\
        0 & \text{if } C = N
    \end{cases} \\
    & V(C) \geq 0.
\end{align}
$$

## Transition Dominance

For a node $i$ and $j$, when the following condition holds, it is known that sweeping $i$ now is at least as good as sweeping $j$ now.

$$
\begin{align}
  & a_i + \sum_{k \in \overline{C}} b_{ik} \leq a_{j} + \sum_{k \in \overline{C}} b_{jk} \\  
  & \sum_{k \in \overline{C}} b_{ik} \leq \sum_{k \in C} b_{ik}.
\end{align}
$$

This knowledge can be modeled as a transition dominance in DIDP.

## Install DIDPPy

In [1]:
!pip install didppy

## Data

In [2]:
# Number of nodes
n = 4
# Node weights
a = [1, 2, 2, 3]
# Edge weights
b = [
    [0, 2, 3, 0],
    [2, 0, 0, 1],
    [3, 0, 0, 2],
    [0, 1, 2, 0],
]

## Modeling

In [3]:
import didppy as dp

model = dp.Model()

node = model.add_object_type(number=n)

# C
clean = model.add_set_var(object_type=node, target=[])

all_nodes = model.create_set_const(object_type=node, value=list(range(n)))
model.add_base_case([clean == all_nodes])

edge_weight = model.add_int_table(b)
# Node weight plus the sum of the edge weights
node_edge_weight = model.add_int_table([a[i] + sum(b[i]) for i in range(n)])
# State function to cache the complement set
contaminated = model.add_set_state_fun(clean.complement())
transition_ids = []

for c in range(n):
    sweep = dp.Transition(
        name="sweep {}".format(c),
        cost=dp.max(
            dp.IntExpr.state_cost(),
            node_edge_weight[c] + edge_weight[clean, contaminated.remove(c)]
        ),
        effects=[(clean, clean.add(c))],
        preconditions=[~clean.contains(c)],
    )
    current_id = model.add_transition(sweep)
    transition_ids.append(current_id)

model.add_dual_bound(0)

In [4]:
# State functions
clean_edge_weights = [
    model.add_int_state_fun(edge_weight[i, clean]) for i in range(n)
]
contaminated_edge_weights = [
    model.add_int_state_fun(edge_weight[i, contaminated]) for i in range(n)
]

# Transition dominance
for i in range(n):
    for j in range(n):
        if i == j:
            continue

        condition1 = a[i] + contaminated_edge_weights[i] <= a[j] + contaminated_edge_weights[j]
        condition2 = contaminated_edge_weights[i] <= clean_edge_weights[i]

        model.add_transition_dominance(
            transition_ids[i],
            transition_ids[j],
            conditions=[condition1, condition2]
        )

## Solving

In [5]:
solver = dp.CABS(model, f_operator=dp.FOperator.Max, quiet=True)
solution = solver.search()

print("Transitions to apply:")
print("")

for t in solution.transitions:
    print(t.name)

print("")
print("Cost: {}".format(solution.cost))

Transitions to apply:

sweep 1
sweep 0
sweep 2
sweep 3

Cost: 8
